## Chapter 23: Linear Algebra

In [1]:
using LinearAlgebra

In [2]:
Matrix(I,3,3)

3×3 Array{Bool,2}:
 1  0  0
 0  1  0
 0  0  1

In [3]:
A = [i+j for i=1:3,j=1:3]

3×3 Array{Int64,2}:
 2  3  4
 3  4  5
 4  5  6

In [4]:
inv(A)

SingularException: SingularException(3)

In [5]:
B = [ 3 2 1; 0 2 0; 1 2 3]

3×3 Array{Int64,2}:
 3  2  1
 0  2  0
 1  2  3

In [6]:
inv(B)

3×3 Array{Float64,2}:
  0.375  -0.25  -0.125
  0.0     0.5   -0.0
 -0.125  -0.25   0.375

In [7]:
A = [0 1 1; 1 2 1; 1 1 -1]
b = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [8]:
x = A \ b

3-element Array{Float64,1}:
 -2.0
  3.0
 -2.0

In [9]:
A*x

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [10]:
T = Tridiagonal([-1 for i=1:4],[2 for j=1:5],[-1 for j=1:4])

5×5 Tridiagonal{Int64,Array{Int64,1}}:
  2  -1   ⋅   ⋅   ⋅
 -1   2  -1   ⋅   ⋅
  ⋅  -1   2  -1   ⋅
  ⋅   ⋅  -1   2  -1
  ⋅   ⋅   ⋅  -1   2

In [11]:
D = Diagonal([1,2,3,4,5])

5×5 Diagonal{Int64,Array{Int64,1}}:
 1  ⋅  ⋅  ⋅  ⋅
 ⋅  2  ⋅  ⋅  ⋅
 ⋅  ⋅  3  ⋅  ⋅
 ⋅  ⋅  ⋅  4  ⋅
 ⋅  ⋅  ⋅  ⋅  5

In [12]:
D*T

5×5 Tridiagonal{Int64,Array{Int64,1}}:
  2  -1   ⋅   ⋅   ⋅
 -2   4  -2   ⋅   ⋅
  ⋅  -3   6  -3   ⋅
  ⋅   ⋅  -4   8  -4
  ⋅   ⋅   ⋅  -5  10

In [13]:
T \ D

5×5 Array{Float64,2}:
 0.833333  1.33333   1.5  1.33333  0.833333
 0.666667  2.66667   3.0  2.66667  1.66667
 0.5       2.0       4.5  4.0      2.5
 0.333333  1.33333   3.0  5.33333  3.33333
 0.166667  0.666667  1.5  2.66667  4.16667

In [14]:
inv(T)*D

5×5 Array{Float64,2}:
 0.833333  1.33333   1.5  1.33333  0.833333
 0.666667  2.66667   3.0  2.66667  1.66667
 0.5       2.0       4.5  4.0      2.5
 0.333333  1.33333   3.0  5.33333  3.33333
 0.166667  0.666667  1.5  2.66667  4.16667

In [15]:
inv(D)*T

5×5 Tridiagonal{Float64,Array{Float64,1}}:
  2.0  -1.0         ⋅          ⋅          ⋅ 
 -0.5   1.0       -0.5         ⋅          ⋅ 
   ⋅   -0.333333   0.666667  -0.333333    ⋅ 
   ⋅     ⋅        -0.25       0.5       -0.25
   ⋅     ⋅          ⋅        -0.2        0.4

In [25]:
inv(D)

5×5 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅         ⋅     ⋅ 
  ⋅   0.5   ⋅         ⋅     ⋅ 
  ⋅    ⋅   0.333333   ⋅     ⋅ 
  ⋅    ⋅    ⋅        0.25   ⋅ 
  ⋅    ⋅    ⋅         ⋅    0.2

In [16]:
methods(det)

# 28 methods for generic function "det":
[1] det(lu::SuiteSparse.UMFPACK.UmfpackLU{Float64,Int32}) in SuiteSparse.UMFPACK at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/SuiteSparse/src/umfpack.jl:392
[2] det(lu::SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}) in SuiteSparse.UMFPACK at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/SuiteSparse/src/umfpack.jl:392
[3] det(lu::SuiteSparse.UMFPACK.UmfpackLU{Complex{Float64},Int64}) in SuiteSparse.UMFPACK at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/SuiteSparse/src/umfpack.jl:399
[4] det(lu::SuiteSparse.UMFPACK.UmfpackLU{Complex{Float64},Int32}) in SuiteSparse.UMFPACK at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/SuiteSparse/src/umfpack.jl:399
[5] det(A::SymTridiagonal; shift) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:440
[6] det(A::Tridiagonal) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/tridiag.jl:742
[7] det(A::UnitUpperTriangular{T,S} where S<:AbstractArray{T,2}) where T in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/triangular.jl:2648
[8] det(A::UnitLowerTriangular{T,S} where S<:AbstractArray{T,2}) where T in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/triangular.jl:2649
[9] det(A::UpperTriangular) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/triangular.jl:2654
[10] det(A::LowerTriangular) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/triangular.jl:2655
[11] det(Q::LinearAlgebra.QRPackedQ) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/qr.jl:922
[12] det(Q::LinearAlgebra.QRCompactWYQ) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/qr.jl:924
[13] det(Q::LinearAlgebra.LQPackedQ) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/lq.jl:342
[14] det(A::Symmetric{var"#s826",S} where S<:(AbstractArray{var"#s8261",2} where var"#s8261"<:var"#s826") where var"#s826"<:Real) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/symmetric.jl:645
[15] det(A::Union{Hermitian{T,S}, Hermitian{Complex{T},S}, Symmetric{T,S}} where S where T<:Real) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/symmetric.jl:644
[16] det(A::Symmetric) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/symmetric.jl:646
[17] det(D::Diagonal) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/diagonal.jl:563
[18] det(F::UpperHessenberg; shift) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/hessenberg.jl:230
[19] det(A::AbstractArray{T,2}) where T in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/generic.jl:1533
[20] det(x::Number) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/generic.jl:1540
[21] det(A::Eigen) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/eigen.jl:424
[22] det(C::Cholesky) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/cholesky.jl:553
[23] det(C::CholeskyPivoted) in Line